In [1]:
import requests
from ast import literal_eval
from utils import ExtractedContent
import pandas as pd
from tqdm.notebook import tqdm
from utils import get_title

In [3]:
def get_fake_user_queries(document_title: str, chapter: str) -> list[str]:
    api_url = 'http://localhost:1234/v1/chat/completions'
    headers = {
        "Content-Type": "application/json",
    }
    data = {
        "model": "gemma-2-9b-it",
        "messages": [
            {"role": "user",
             "content": f"Du bist ein Mediziner, der in der Notaufname arbeitet und suchst Informationen aus dem Kapitel {chapter} aus der Leitlinie {document_title}, verfasse 10 realistische Sucheingaben um relevante Informationen aus dem Kapitel zu extrahieren, du suchst nur die Informationen zu deinem Fall, schreibe die Suchanfragen, als ob du die Namen der Leitlinie und des Kapitels nicht kennen würdest, verwende in den Anfragen verschiedene Ausdrücke für die Umstände des Patienten"},
        ],
        "temperature": 0,
        "max_tokens": -1,
        "stream": False}
    response = requests.post(api_url, headers=headers, json=data)

    # Antwort verarbeiten
    if response.status_code == 200:
        result = response.json()
        results = list(literal_eval(result['choices'][0]['message']['content']).values())
    else:
        print("Fehler:", response.status_code, response.text)
        results = []
    return results

In [4]:
list(ExtractedContent.contents['040-014l_S3_Intensivmedizin_nach_Polytrauma_2024-08.pdf'.lower()].keys())

['HERAUSGEBENDE',
 'DIE WICHTIGSTEN EMPFEHLUNGEN AUF EINEN BLICK',
 'GELTUNGSBEREICH UND ZWECK',
 'A. ZIELSETZUNG UND FRAGESTELLUNG',
 'B. VERSORGUNGSBEREICH',
 'C. PATIENT*INNENZIELGRUPPE',
 'D. ADRESSATEN',
 'E. WEITERE DOKUMENTE ZU DIESER LEITLINIE',
 'ZUSAMMENSETZUNG DER LEITLINIENGRUPPE',
 'LEITLINIENKOORDINATION/ANSPRECHPARTNER',
 'BETEILIGTE FACHGESELLSCHAFTEN UND ORGANISATION',
 'KAPITELAUTOREN',
 'VORWORT',
 '1 INTERDISZIPLINÄRE UND INTERPROFESSIONELLE ZUSAMMENARBEIT UND KOMMUNIKATION',
 '1.1 INTERDISZIPLINÄRE UND INTERPROFESSIONELLE ZUSAMMENARBEIT UND KOMMUNIKATION PRINZIPIEN DER ZUSAMMENARBEIT IN DER POLYTRAUMAVERSORGUNG',
 '1.2 INTERDISZIPLINÄRE UND INTERPROFESSIONELLE ZUSAMMENARBEIT UND KOMMUNIKATION OPTIMIERTES SETTING FÜR ZUSAMMENARBEIT UND KOMMUNIKATION',
 '1.3 INTERDISZIPLINÄRE UND INTERPROFESSIONELLE ZUSAMMENARBEIT UND KOMMUNIKATION BETEILIGE FACHDISZIPLINEN UND PROFESSIONEN IN DER INTENSIVMEDIZINISCHEN POLYTRAUMA-VERSORGUNG',
 '1.4 INTERDISZIPLINÄRE UND INTERPROFESSI

In [5]:
excluded_chapters = ["Leitsatz", "Inhaltsverzeichnis", "Literaturverzeichnis",
                     "E. Ökonomie, Qualitätssicherung und Implementierung der Leitlinie",
                     "A. Analgesie, Sedierung und Delirmanagement bei Erwachsenen", "B. Spezielle Patient:innengruppen",
                     "C. Analgesie, Sedierung und Delirmanagement bei Kindern",
                     "D. Analgesie, Sedierung und Delirmanagement bei älteren Patient:innen",
                     "Informationen zu dieser Leitlinie", '2. AUFNAHME AUF DIE INTENSIVSTATION',
                     'Informationen zu dieser Leitlinie - 2.1 Federführende Fachgesellschaft', 'III. ANLAGEN',
                     '3. MONITORING', 'HERAUSGEBENDE', 'DIE WICHTIGSTEN EMPFEHLUNGEN AUF EINEN BLICK',
                     'GELTUNGSBEREICH UND ZWECK', 'A. ZIELSETZUNG UND FRAGESTELLUNG', 'B. VERSORGUNGSBEREICH',
                     'C. PATIENT*INNENZIELGRUPPE', 'D. ADRESSATEN', 'E. WEITERE DOKUMENTE ZU DIESER LEITLINIE',
                     'ZUSAMMENSETZUNG DER LEITLINIENGRUPPE', 'LEITLINIENKOORDINATION/ANSPRECHPARTNER',
                     'BETEILIGTE FACHGESELLSCHAFTEN UND ORGANISATION', 'KAPITELAUTOREN', 'VORWORT', 'KOMMENTARLISTE',
                     '14. SCHMERZMANAGEMENT',
                     '9.2 LAGERUNG UND PHYSIOTHERAPIE LAGERUNG BEI SPEZIELLEN KRANKHEITSBILDERN',
                     'SYSTEMATIK DER LEITLINIE', '11. WUNDMANAGEMENT', '5. BEATMUNG',
                     '4. BILDGEBUNG IM STATIONÄREN VERLAUF', 'ERLÄUTERUNG DER ABKÜRZUNGEN',
                     '10. MULTIORGANVERSAGEN UND ORGANERSATZTHERAPIE',
                     '1 INTERDISZIPLINÄRE UND INTERPROFESSIONELLE ZUSAMMENARBEIT UND KOMMUNIKATION', 'LITERATUR',
                     '15. DELIR- UND SEDIERUNGSMANAGEMENT', '9. LAGERUNG UND PHYSIOTHERAPIE',
                     'Informationen zu dieser Leitlinie - 2.2 Kontakt', '7. DEFINITIVE CHIRURGISCHE VERSORGUNG',
                     'Informationen zu dieser Leitlinie - 2.3 Zitierweise', '6. VOLUMEN- UND KREISLAUFTHERAPIE',
                     '4.5.1 BILDGEBUNG IM STATIONÄREN VERLAUF KRANIO-ZERVIKO-SPINALE VERLETZUNGEN',
                     'Informationen zu dieser Leitlinie - 2.4 Besonderer Hinweis',
                     'Informationen zu dieser Leitlinie - 2.5 Verantwortlichkeiten',
                     'Informationen zu dieser Leitlinie - 2.5.1 Beteiligte Fachgesellschaften und Organisationen',
                     'Informationen zu dieser Leitlinie - 2.5.2 Autoren dieser Leitlinie und Mitglieder der Leitliniengruppe',
                     'Informationen zu dieser Leitlinie - 2.5.3 Methodische Begleitung',
                     'Informationen zu dieser Leitlinie - 2.5.4 Weitere Autoren und Fachgesellschaften dieser Leitlinie und Mitglieder der Leitliniengruppe (kein Stimmrecht)',
                     'Informationen zu dieser Leitlinie - 2.5.5 Beteiligte Fachgesellschaften ohne aktive Mitarbeit in der Leitliniengruppe',
                     'Informationen zu dieser Leitlinie - 2.6 Zielsetzung', 'Klinische Erstversorgung',
                     'Informationen zu dieser Leitlinie - 2.7 Grundlagen der Methodik',
                     'Informationen zu dieser Leitlinie - 2.7.1 Verwendete Definitionen für Empfehlungs- und Konsensstärken',
                     'Informationen zu dieser Leitlinie - 2.7.2 Gültigkeitsdauer und Aktualisierungsverfahren',
                     'Anhang', 'Rettung', 'Symptome und Diagnostik',
                     'Ätiologie und Pathogenese der Kohlenmonoxidvergiftung', 'Anhang', 'Qualitätssicherung']

In [6]:
test_data = {'query': [], 'document_title': [], 'chapter_name': []}
for filename, table_of_contents in tqdm(ExtractedContent.contents.items(), leave=False):
    title = get_title(filename)
    for chapter in tqdm(table_of_contents, leave=False):
        if chapter not in excluded_chapters:
            queries = get_fake_user_queries(title, chapter)
            for query in queries:
                test_data['query'].append(query)
                test_data['document_title'].append(title)
                test_data['chapter_name'].append(chapter)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

In [16]:
test_df = pd.DataFrame(test_data)
test_df

,query,document_title,chapter_name
0,[Schmerztherapie bei bewusstlosem Patienten im...,s3 analgesie sedierung delirmanagement in der ...,"A. Analgesie, Sedierung und Delirmanagement be..."
1,[Risikofaktoren für Schmerzen bei Intensivpati...,s3 analgesie sedierung delirmanagement in der ...,"A. Analgesie, Sedierung und Delirmanagement be..."
2,[Prävention von Schmerzen bei Intensivpatiente...,s3 analgesie sedierung delirmanagement in der ...,"A. Analgesie, Sedierung und Delirmanagement be..."
3,[Prävention akuter Belastungen bei Intensivpat...,s3 analgesie sedierung delirmanagement in der ...,"A. Analgesie, Sedierung und Delirmanagement be..."
4,[Monitoring der Schmerztherapie bei einem bewu...,s3 analgesie sedierung delirmanagement in der ...,"A. Analgesie, Sedierung und Delirmanagement be..."
...,...,...,...
122,[Frühzeitige Rehabilitation nach Polytrauma im...,s3 intensivmedizin nach polytrauma,16. FRÜHZEITIGES REHABILITATIONSORIENTIERTES E...
123,[Behandlungsprioritäten bei polytraumatisierte...,s3 intensivmedizin nach polytrauma,17. KAPITELÜBERGREIFENDE GENDER- UND ALTERSSPE...
124,[Kosten-Nutzen-Analyse bei polytraumatisierten...,s3 intensivmedizin nach polytrauma,18. KOSTENNUTZEN
125,[Komplikationen nach Polytrauma im Intensivber...,s3 intensivmedizin nach polytrauma,19. FEHLERMANAGEMENT UND VERMEIDUNG VON KOMPLI...


In [17]:
test_df = test_df.explode('query').reset_index(drop=True)
test_df.shape

(1270, 3)

In [18]:
test_df.to_csv('data/evaluation/synthetic_queries.csv')